# Pre-processing Choppers

In [ ]:
import scipp as sc
import matplotlib.pyplot as plt

In [ ]:
# TODO remove
%matplotlib widget

This guide uses fake data for a chopper which roughly represents what a real chopper loaded from NeXus looks like.
ScippNeutron has a function for generating this data:

In [ ]:
from scippneutron.data import chopper_mockup

chopper = chopper_mockup()
chopper

We can see that there is some information about the slits and geometry of the chopper as well as some timing-related data.
Take a look at the [NXdisk_chopper](https://manual.nexusformat.org/classes/base_classes/NXdisk_chopper.html) documentation for an overview of the fields.

Some fields are nested data groups which appear when a NeXus file contains `NXlog`s.
We can extract the relevant arrays from them using `post_process_disk_chopper`:

In [ ]:
from scippneutron.chopper import post_process_disk_chopper

chopper = post_process_disk_chopper(chopper)
chopper

Some data varies with time.
This is difficult to deal with, e.g., when [unwrapping frames](./frame-unwrapping.ipynb).
So we need to extract time-independent quantities from the raw chopper data.

## Identify in-phase regions

Frame unwrapping is only feasible when the chopper is in-phase with the neutron source as wavelength ranges and resolutions would change with time otherwise.
To identify regions where the chopper is in-phase, we first find plateaus in the `rotation_speed` which is the rotation frequency of the chopper.

In [ ]:
rotation_speed = chopper['rotation_speed']
rotation_speed.name = 'rotation_speed'
rotation_speed

The chopper has a long region of near-constant rotation speed surrounded by spin-up and spin-down regions:

In [ ]:
rotation_speed.plot(markersize=2)

Use [find_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) and [collapse_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) to find those plateaus.
Note the `atol` and `min_n_points` parameters which need to be tuned for specific data.

In [ ]:
from scippneutron.chopper import collapse_plateaus, find_plateaus

plateaus = find_plateaus(rotation_speed, atol=sc.scalar(1e-3, unit='Hz'), min_n_points=10)
plateaus = collapse_plateaus(plateaus)
plateaus

In this case, the `find_plateaus` found two plateaus that we can plot with the following helper function:

In [ ]:
def plot_plateaus(raw_data: sc.DataArray, plateaus: sc.DataArray) -> None:
    fig, ax = plt.subplots(1)
    raw_data.plot(ax=ax, markersize=2)
    for plateau in plateaus:
        i = plateau.coords['plateau'].value
        da = sc.DataArray(
            plateau.data.broadcast(dims=['time'], shape=[2]),
            coords={'time': plateau.coords['time']},
            name=f'Plateau {i}')
        da.plot(ax=ax, ls='-', marker='|', c=f'C{i + 1}')

In [ ]:
plot_plateaus(rotation_speed, plateaus)

In this case, the source has a frequency of 14Hz which means that plateau 0 is in phase.
But plateau 1 is not, it is a short region where the chopper slowed down before fully stopping.

We can use [filter_in_phase](../../generated/modules/scippneutron.chopper.filtering.filter_in_phase.rst) to remove all out-of-phase plateaus:

In [ ]:
source_frequency = sc.scalar(14.0, unit='Hz')

In [ ]:
from scippneutron.chopper import filter_in_phase

filtered = filter_in_phase(plateaus,
                           reference=source_frequency,
                           rtol=sc.scalar(1e-3))
filtered

In [ ]:
plot_plateaus(rotation_speed, filtered)